# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 71828 records between (datetime.datetime(2021, 2, 3, 0, 0), datetime.datetime(2021, 2, 3, 0, 0)).
Subsetted dataframe with 50000 records.
Record num : 50000
Wall time: 15.6 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)
        continue

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    data = get_trend_fwd(data)
#     data = trend_angle_check(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = calculate_pl(data)
    data = simple_take_profit(data)
    data = simple_stop_loss(data)
    data = make_order(data)

100%|███████████████████████████████████| 50000/50000 [04:34<00:00, 182.34it/s]


In [4]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 50000
Hours     : 15.16
Recs/hour : 3298
Recs/min  : 54
Recs/sec  : 0.9


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,...,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l,long_open,long_close,short_open,short_close,pl
0,20210203 00:00:00.267,1.20365,1.20371,1.203680,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210203 00:00:00.419,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210203 00:00:00.723,1.20368,1.20370,1.203690,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210203 00:00:00.979,1.20366,1.20371,1.203685,2021-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210203 00:00:01.182,1.20367,1.20371,1.203690,2021-02-03 00:00:01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pl_list = data['df']['pl'][data['df']['pl'].notnull()]
print(pl_list.value_counts())
print(sum(pl_list))

 0.001    2
-0.001    1
Name: pl, dtype: int64
0.001


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [9]:
send_telegram_message(f'Run Complete at : {dt.datetime.now()}')